### IMPORTING PACAKGES

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords

import string

data = pd.read_csv("/content/netflixData.csv")
# print(data.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
### chceking weather any null values are there
print(data.isnull().sum())

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


In [3]:
### viewing columns in the datset
data.columns

Index(['Show Id', 'Title', 'Description', 'Director', 'Genres', 'Cast',
       'Production Country', 'Release Date', 'Rating', 'Duration',
       'Imdb Score', 'Content Type', 'Date Added'],
      dtype='object')

In [4]:
### seeing the columns in the datset
data = data[["Title", "Description", "Content Type", "Genres","Imdb Score"]]
print(data.head())

                           Title  \
0                       (Un)Well   
1                         #Alive   
2  #AnneFrank - Parallel Stories   
3                       #blackAF   
4               #cats_the_mewvie   

                                         Description Content Type  \
0  This docuseries takes a deep dive into the luc...      TV Show   
1  As a grisly virus rampages a city, a lone man ...        Movie   
2  Through her diary, Anne Frank's story is retol...        Movie   
3  Kenya Barris and his family navigate relations...      TV Show   
4  This pawesome documentary explores how our fel...        Movie   

                                           Genres Imdb Score  
0                                      Reality TV     6.6/10  
1  Horror Movies, International Movies, Thrillers     6.2/10  
2             Documentaries, International Movies     6.4/10  
3                                     TV Comedies     6.6/10  
4             Documentaries, International Movies    

In [5]:
### viewing first 10 rows
data.head(10)

Title  \
0                       (Un)Well   
1                         #Alive   
2  #AnneFrank - Parallel Stories   
3                       #blackAF   
4               #cats_the_mewvie   
5              #FriendButMarried   
6            #FriendButMarried 2   
7                   #realityhigh   
8                      #Rucker50   
9                        #Selfie   

                                         Description Content Type  \
0  This docuseries takes a deep dive into the luc...      TV Show   
1  As a grisly virus rampages a city, a lone man ...        Movie   
2  Through her diary, Anne Frank's story is retol...        Movie   
3  Kenya Barris and his family navigate relations...      TV Show   
4  This pawesome documentary explores how our fel...        Movie   
5  Pining for his high school crush for years, a ...        Movie   
6  As Ayu and Ditto finally transition from best ...        Movie   
7  When nerdy high schooler Dani finally attracts...        Movie   
8  This documentary celebrates the 50th anniversa...        Movie   
9  Two days before their final exams, three teen ...        Movie   

                                           Genres Imdb Score  
0                                      Reality TV     6.6/10  
1  Horror Movies, International Movies, Thrillers     6.2/10  
2             Documentaries, International Movies     6.4/10  
3                                     TV Comedies     6.6/10  
4             Documentaries, International Movies     5.1/10  
5   Dramas, International Movies, Romantic Movies     7.0/10  
6   Dramas, International Movies, Romantic Movies     7.0/10  
7                                        Comedies     5.1/10  
8                    Documentaries, Sports Movies     5.1/10  
9          Comedies, Dramas, International Movies     5.8/10

In [6]:
### droping null values
data = data.dropna()

In [7]:
### seeing the columns that contains null values
data.isnull().sum()

Title           0
Description     0
Content Type    0
Genres          0
Imdb Score      0
dtype: int64

In [8]:
### preprocessing
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Title"] = data["Title"].apply(clean)

In [9]:
print(data.Title.sample(10))

3237                      mujer arriba
975                       children sea
4720                claudia kishi club
3397              nicki jam el ganador
1129                             creep
81                                 day
2968    mariah carey merriest christma
113                        heavi heart
2403                        john henri
666        bill burr im sorri feel way
Name: Title, dtype: object


In [10]:
genre_list = data['Genres'].tolist()

tfidf_vectorizer = TfidfVectorizer(stop_words='english', input='content')

# Fit and transform the genre list using TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(genre_list)

# Calculate the cosine similarity matrix
similarity = cosine_similarity(tfidf_matrix)

In [11]:
indices = pd.Series(data.index,
                    index=data['Title']).drop_duplicates()

In [12]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return data['Title'].iloc[movieindices]

print(netFlix_recommendation("fall"))

47       türken  ein babi
61          star christma
63                       
114             home view
142           remark tale
158          trip jamaica
253    alarmoti land fire
338          among famili
357     andaleeb el dokki
435             asu mare 
Name: Title, dtype: object


### 1. Find out the list of most popular and liked genre

In [13]:
# Function to convert IMDb score strings to numeric values
def convert_imdb_score(score):
    try:
        return float(score.split('/')[0])  # Extract the numeric part before "/"
    except:
        return 0.0  # Default value for invalid scores

# Convert IMDb score strings to numeric values
data['Imdb Score'] = data['Imdb Score'].apply(convert_imdb_score)

# Calculate average IMDB score per genre
avg_scores_by_genre = data.groupby('Genres')['Imdb Score'].mean()

# Identify most popular genres based on average IMDB scores
most_popular_genres = avg_scores_by_genre.nlargest(5)

# Identify liked genres with more movies and above-average scores
above_avg_scores = avg_scores_by_genre[avg_scores_by_genre >= avg_scores_by_genre.mean()]
liked_genres = above_avg_scores.nlargest(5)

print("Most Popular Genres and like genres")
print(most_popular_genres)

Most Popular Genres and like genres
Genres
Classic & Cult TV, Kids' TV, TV Action & Adventure                        9.300000
British TV Shows, Classic & Cult TV, International TV Shows               8.733333
TV Action & Adventure, TV Horror, TV Sci-Fi & Fantasy                     8.600000
Action & Adventure, Classic Movies, Sci-Fi & Fantasy                      8.500000
British TV Shows, International TV Shows, Stand-Up Comedy & Talk Shows    8.500000
Name: Imdb Score, dtype: float64


### 2.Create Model that finds the best suited Movie for one user in every genre

In [14]:
data=pd.read_csv("/content/netflixData.csv")
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english', input='content')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Genres'])

# Create a dictionary to map genres to movies
genres_to_movies = {}
for idx, genre in enumerate(data['Genres']):
    if genre not in genres_to_movies:
        genres_to_movies[genre] = []
    genres_to_movies[genre].append(idx)

# Function to extract numeric part from "IMDB Score" string
def extract_numeric_part(score):
    if isinstance(score, str):
        match = re.search(r'(\d+\.\d+)', score)
        if match:
            return float(match.group(1))
    return 0.0

# For each genre, recommend the movie with the highest IMDB Score
for genre, movie_indices in genres_to_movies.items():
    max_imdb_score = -1
    recommended_movie_idx = None

    for idx in movie_indices:
        imdb_score = data.loc[idx, 'Imdb Score']
        numeric_imdb_score = extract_numeric_part(imdb_score)

        if numeric_imdb_score > max_imdb_score:
            max_imdb_score = numeric_imdb_score
            recommended_movie_idx = idx

    recommended_movie_title = data.loc[recommended_movie_idx, 'Title']
    print(f"Recommended movie in {genre}: {recommended_movie_title} (IMDB Score: {max_imdb_score:.2f})")

Recommended movie in Reality TV: Car Masters: Rust to Riches (IMDB Score: 8.80)
Recommended movie in Horror Movies, International Movies, Thrillers: The Blue Elephant 2 (IMDB Score: 8.20)
Recommended movie in Documentaries, International Movies: Unbroken (IMDB Score: 8.80)
Recommended movie in TV Comedies: Chappelle's Show (IMDB Score: 8.80)
Recommended movie in Dramas, International Movies, Romantic Movies: Qismat (IMDB Score: 8.40)
Recommended movie in Comedies: Parrot Sketch Not Included: Twenty Years of Monty Python (IMDB Score: 8.00)
Recommended movie in Documentaries, Sports Movies: Roll With Me (IMDB Score: 8.40)
Recommended movie in Comedies, Dramas, International Movies: No Longer kids (IMDB Score: 9.00)
Recommended movie in Comedies, International Movies, Romantic Movies: The Married Couples (IMDB Score: 8.10)
Recommended movie in International TV Shows, Romantic TV Shows, TV Dramas: Fifty: The Series (IMDB Score: 8.40)
Recommended movie in Docuseries, Science & Nature TV: Ou

### 3. Find what Genre Movies have received the best and worst ratings based on User Rating.

In [15]:
def best_worst_rated_genres():
    # Extract relevant columns from data

    ratings_data = data[['Genres', 'Imdb Score']]

    # Extract numeric part of IMDB scores and convert to numeric
    ratings_data['Imdb Score'] = ratings_data['Imdb Score'].str.extract(r'(\d+\.\d+)').astype(float)

    # Group by genres and calculate average IMDB scores
    avg_scores_by_genre = ratings_data.groupby('Genres')['Imdb Score'].mean()

    # Drop genres with NaN or empty IMDB scores
    avg_scores_by_genre = avg_scores_by_genre.dropna()

    if avg_scores_by_genre.empty:
        return None, None

    # Find best and worst rated genres
    best_rated_genre = avg_scores_by_genre.idxmax()
    worst_rated_genre = avg_scores_by_genre.idxmin()

    return best_rated_genre, worst_rated_genre

# Call the function to get the best and worst rated genres
best_genre, worst_genre = best_worst_rated_genres()

if best_genre is None or worst_genre is None:
    print("No valid IMDB scores found.")
else:
    print("Best Rated Genre:", best_genre)
    print("Worst Rated Genre:", worst_genre)


Best Rated Genre: Classic & Cult TV, Kids' TV, TV Action & Adventure
Worst Rated Genre: Action & Adventure, Cult Movies, Sci-Fi & Fantasy


<ipython-input-15-af2f0e431937>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_data['Imdb Score'] = ratings_data['Imdb Score'].str.extract(r'(\d+\.\d+)').astype(float)
